# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [31]:
import nltk
nltk.download(['punkt', 'wordnet','omw-1.4'])

[nltk_data] Downloading package punkt to /Users/az/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/az/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/az/nltk_data...


True

In [99]:
# import libraries
import re
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer 
from sklearn.multioutput import MultiOutputClassifier 

import pickle

In [42]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('InsertTableName', engine)
X = df['message']   # id, message, original, genre
Y = df.iloc[:, 4:]

### 2. Write a tokenization function to process your text data

In [43]:
def tokenize(text):
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()    #considering removing stop words
    return [lemmatizer.lemmatize(tok).lower().strip() for tok in tokens]

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [45]:
pipeline = Pipeline([
    ('textpipeline', Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('Tfidf', TfidfTransformer())
    ])),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [46]:
from random import random

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, random_state=42)
pipeline.fit(X_train, Y_train)

Pipeline(steps=[('textpipeline',
                 Pipeline(steps=[('vect',
                                  CountVectorizer(tokenizer=<function tokenize at 0x7ff70a4133a0>)),
                                 ('Tfidf', TfidfTransformer())])),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [50]:
Y_pred = pipeline.predict(X_test)

In [84]:
category_names = df.columns[4:]
reports = dict()
for i, cat in enumerate(category_names):
    reports[cat] = classification_report(Y_test[cat], Y_pred[:, i], zero_division=0, output_dict=True)
accuracies = [reports[report]['accuracy'] for report in reports]
print(np.array(accuracies).mean())

0.9454525607438229


### 6. Improve your model
Use grid search to find better parameters. 

In [94]:
parameters = {
        'textpipeline__vect__ngram_range': ((1, 1), (1, 2)),
        'clf__estimator__n_estimators': [50, 100, 200],
        'clf__estimator__min_samples_split': [2, 3, 4]
    }

cv = GridSearchCV(pipeline, parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [95]:
cv.fit(X_train, Y_train)
Y_pred = cv.predict(X_test) # run for ~6h

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [98]:
cv.best_params_

{'clf__estimator__min_samples_split': 2,
 'clf__estimator__n_estimators': 100,
 'textpipeline__vect__ngram_range': (1, 2)}

### 9. Export your model as a pickle file

In [100]:
pickle.dump(cv, open('../models/classifier.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.